### 1. Advanced CRUD Operations
- **Exercise 1**: Use `bulk_write` to insert, update, and delete multiple documents simultaneously.
- **Task 1**: Insert a dataset of `courses` where each course has multiple student enrollments, then display the data.


In [1]:
# Import beberapa module seperti : 
# MongoCLient untuk melakukan koneksi dengan MongoDB
# InsertOne, UpdateOne, DeleteOne salah satu fungsi bulk_write untuk menjalankan banyak operasi dalam satu waktu
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne, DeleteMany, UpdateMany

# Import mmodule library pandas
import pandas as pd

# Melakukan koneksi pada MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Membuat sebuah database bernama university_db
db = client['university_db']

# Membuat sebuah collection bernama courses yang digunakan pada variabel courses_collection
courses_collection = db['courses']

# Agar saat dijalankan ulang data tidak semakin banyak
courses_collection.delete_many({})

# Membuat beberapa operasi yang nantinya akan di eksekusi secara bersamaan
operations = [
    # Menambahkan isi dari data agar lebih banyak
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 202', 'enrollments': 28, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 303', 'enrollments': 35, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 203', 'enrollments': 40, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 304', 'enrollments': 32, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 405', 'enrollments': 27, 'department': 'Computer Science'}),
    InsertOne({'course': 'Physics 201', 'enrollments': 22, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 18, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 303', 'enrollments': 26, 'department': 'Physics'}),
    InsertOne({'course': 'Chemistry 101', 'enrollments': 24, 'department': 'Chemistry'}),
    InsertOne({'course': 'Chemistry 202', 'enrollments': 31, 'department': 'Chemistry'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'History 302', 'enrollments': 17, 'department': 'History'}),
    InsertOne({'course': 'Literature 101', 'enrollments': 19, 'department': 'Literature'}),
    InsertOne({'course': 'Literature 203', 'enrollments': 23, 'department': 'Literature'}),
    InsertOne({'course': 'Economics 101', 'enrollments': 29, 'department': 'Economics'}),
    InsertOne({'course': 'Economics 202', 'enrollments': 33, 'department': 'Economics'}),
    InsertOne({'course': 'Engineering 101', 'enrollments': 21, 'department': 'Engineering'}),
    InsertOne({'course': 'Engineering 202', 'enrollments': 25, 'department': 'Engineering'}),

    # Menghapus data, yang jumlah datanya kurang dari 20
    DeleteMany(
        {"enrollments" :  { "$lt" : 20} }
    ),

    # Memperbari seluruh departement Engineering menjadi Unformation Technology
    UpdateMany(
        filter={ 'department' : 'Engineering' },
        update={ '$set' : { 'department' : 'Information Technology' } }
    )

]

# Menjalankan method bulk_write agar banyak operasi dapat dijalankan secara bersamaan
courses_collection.bulk_write(operations)
# JIka berhasil jalankan oerintah ini
print('Courses inserted successfully.')

# Menampilkan isi dataset
df = pd.DataFrame(courses_collection.find({}))
df.drop(columns=['_id'])

Courses inserted successfully.


,course,enrollments,department
0,Math 101,30,Mathematics
1,Math 202,28,Mathematics
2,Math 303,35,Mathematics
3,CS 102,25,Computer Science
4,CS 203,40,Computer Science
5,CS 304,32,Computer Science
6,CS 405,27,Computer Science
7,Physics 201,22,Physics
8,Physics 303,26,Physics
9,Chemistry 101,24,Chemistry


### 2. Complex Filtering and Querying
- **Exercise 2**: Filter courses with enrollments over 20 students.
- **Task 2**: Retrieve and display courses only from the 'Computer Science' or 'Mathematics' departments.


In [2]:
# Memfiltering courses yang di atas 20 pendaftaran
df = pd.DataFrame(courses_collection.find({'enrollments' : { '$gt' : 20 }}))
df.drop(columns=['_id'])

,course,enrollments,department
0,Math 101,30,Mathematics
1,Math 202,28,Mathematics
2,Math 303,35,Mathematics
3,CS 102,25,Computer Science
4,CS 203,40,Computer Science
5,CS 304,32,Computer Science
6,CS 405,27,Computer Science
7,Physics 201,22,Physics
8,Physics 303,26,Physics
9,Chemistry 101,24,Chemistry


In [3]:
# Menampilkan dimana terdapat matakuliah matematika dan sains 
df = pd.DataFrame(courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}))
df.drop(columns=['_id'])

,course,enrollments,department
0,Math 101,30,Mathematics
1,Math 202,28,Mathematics
2,Math 303,35,Mathematics
3,CS 102,25,Computer Science
4,CS 203,40,Computer Science
5,CS 304,32,Computer Science
6,CS 405,27,Computer Science


### 3. Aggregation Framework for Data Analysis
- **Exercise 3**: Use aggregation to find the average enrollment per department.
- **Task 3**: Create an aggregation pipeline that finds the maximum enrollment for each department.


In [4]:
# Membuat sebuah pipeline d untuk menemukan rata - rata pendaftaran per departemen
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]

# Menampilkan sekaligus menjalankan fungsi pipeline pada agregasi
df = pd.DataFrame(list(courses_collection.aggregate(pipeline)))
df

,_id,average_enrollment
0,Information Technology,23.0
1,Chemistry,27.5
2,Physics,24.0
3,Mathematics,31.0
4,Computer Science,31.0
5,History,20.0
6,Literature,23.0
7,Economics,31.0


In [5]:
# Membuat sebuah pipeline d untuk menemukan rata - rata pendaftaran per departemen
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]

# Menampilkan sekaligus menjalankan fungsi pipeline pada agregasi
df = pd.DataFrame(list(courses_collection.aggregate(pipeline)))
df

,_id,max_enrollment
0,Information Technology,25
1,Chemistry,31
2,Physics,26
3,Mathematics,35
4,Computer Science,40
5,History,20
6,Literature,23
7,Economics,33


### 4. Data Transformation using project and $addFields
- **Exercise 4**: Use `$project` to rename and only show fields: `course_name`, `department_name`, and `enrollments`.
- **Task 4**: Use `$addFields` to create a new field `enrollment_category` where enrollments > 20 are 'high' and others 'low'.


In [6]:
# Membuat sebuah pipeline hanya untuk menampilkan couse_name dan departemen_name dan enrollment
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]

# Menampilkan sekaligus menjalankan fungsi pipeline pada agregasi
df = pd.DataFrame(list(courses_collection.aggregate(pipeline)))
df.drop(columns=['_id'])

,enrollments,course_name,department_name
0,30,Math 101,Mathematics
1,28,Math 202,Mathematics
2,35,Math 303,Mathematics
3,25,CS 102,Computer Science
4,40,CS 203,Computer Science
5,32,CS 304,Computer Science
6,27,CS 405,Computer Science
7,22,Physics 201,Physics
8,26,Physics 303,Physics
9,24,Chemistry 101,Chemistry


In [7]:
# Membuat sebuah pipeline untuk menambah kolom baru, dimana colom baru bernama enrollment_category yang kondisinya apabila berdasarkan enrollment
# apabila enrollments lebih besar dari 20 maka isinya high kalau lebih kecil maka isinya low
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]

# Menampilkan sekaligus menjalankan fungsi pipeline pada agregasi
df = pd.DataFrame(list(courses_collection.aggregate(pipeline)))
df.drop(columns=['_id'])

,course,enrollments,department,enrollment_category
0,Math 101,30,Mathematics,high
1,Math 202,28,Mathematics,high
2,Math 303,35,Mathematics,high
3,CS 102,25,Computer Science,high
4,CS 203,40,Computer Science,high
5,CS 304,32,Computer Science,high
6,CS 405,27,Computer Science,high
7,Physics 201,22,Physics,high
8,Physics 303,26,Physics,high
9,Chemistry 101,24,Chemistry,high


### Homework for Students
- **Homework 1**: Perform an aggregation to get a count of courses per department.

In [8]:
# Membuat sebuah pipeline untuk menghitung jumlah kursus per departement
pipeline = [
    
    { '$group': { 
        # Langkah pertama lakukan grouping terlebih dahulu bedasarkan departemen
        '_id': '$department',
        # Setelah dilakukan grouping maka di hitung total jumlahnya dari isi departement
        'count_courses' : { '$sum' : 1 }
        } 
    },
    { '$project' : {
        # Kemudian tinggal menampilkan dalam bentuk yang kita inginkan
        'deparment' : '$department',
        'count_courses' : '$count_courses'
        }
    }
]

# Menampilkan sekaligus menjalankan fungsi pipeline pada agregasi
df = pd.DataFrame(list(courses_collection.aggregate(pipeline)))
df

,_id,count_courses
0,Information Technology,2
1,Chemistry,2
2,Computer Science,4
3,Mathematics,3
4,Physics,2
5,History,1
6,Economics,2
7,Literature,1


### Homework for Students
- **Homework 2**: Use `$match` and `$group` together to filter and get only courses with enrollments over 25 in 'Computer Science'.


In [9]:
# Membuat sebuah pipeline untuk memendapatkan enrolment yang lebih besar dari 25 di computer science
pipeline = [
    # Lakukan filterisasi dimana hanya department yang berasal dari computer science saja
    {
      '$match' : {'department':'Computer Science'	}  
    },
    # Lalu bentuk terlebih dahulu output yang akan di goruping
    { '$project' : {
        # Kemudian tinggal menampilkan dalam bentuk yang kita inginkan
        'department' : '$department',
        # Kemudian hanya menampilkan yang berada di atas pendaftaran 25 orang saja
        'courses_name': {
            '$cond': {
                'if': { '$gt': ['$enrollments', 25] },
                'then': '$course',
                'else': None
            }
        }
    }},
    # Dari bentuk di atas tinggal di grouping saja
    # Lalu masukan semua data courses_name pada courses
    { '$group' : {
        '_id' : '$department',
        'courses' : { '$push': '$courses_name' }
        }
    }
]

# Menampilkan sekaligus menjalankan fungsi pipeline pada agregasi
df = pd.DataFrame(list(courses_collection.aggregate(pipeline)))
print(list(courses_collection.aggregate(pipeline)))

df

[{'_id': 'Computer Science', 'courses': [None, 'CS 203', 'CS 304', 'CS 405']}]


,_id,courses
0,Computer Science,"[None, CS 203, CS 304, CS 405]"


### Homework for Students
- **Homework 3**: Apply `$lookup` to join `courses` collection with `students` collection based on student enrollments.


In [13]:
# Membuat collection baru yaitu students yang akan melakukan join pada course
students_collection = db['students']
students_collection.delete_many({})  # reset dulu

# MEnyiapkan data tersebut
students_data = [
    {'student_id': 1, 'name': 'Julioez', 'course': 'Math 101'},
    {'student_id': 2, 'name': 'Candita', 'course': 'CS 203'},
    {'student_id': 3, 'name': 'Haga', 'course': 'Physics 201'},
    {'student_id': 4, 'name': 'Figo', 'course': 'Engineering 202'},
    {'student_id': 5, 'name': 'Latupeirissa', 'course': 'Economics 202'},
]

# Lalu dari data yang disiapkan tersebut kemudian di tambahkan kedalam collection
students_collection.insert_many(students_data)
df = pd.DataFrame(students_collection.find({}))
df
    

,_id,student_id,name,course
0,691346d51e2873a74577e115,1,Julioez,Math 101
1,691346d51e2873a74577e116,2,Candita,CS 203
2,691346d51e2873a74577e117,3,Haga,Physics 201
3,691346d51e2873a74577e118,4,Figo,Engineering 202
4,691346d51e2873a74577e119,5,Latupeirissa,Economics 202


In [14]:
pipeline = [
    {
        "$lookup": {
            # Nama collection yang mau di-join
            "from": "students",             

            # Ini seperti field pada courses yang memiliki kesamaan atau relasi pada field di student
            # Pada sql seperti courses.course = students.course 
            "localField": "course",         # Field di collection courses
            "foreignField": "course",       # Field di collection students
            "as": "student_details"         # Hasil join disimpan di sini
        }
    },
    # Gunakan match untuk melakukan filter hanya menammpilkan yang memiliki relasi dsaja
    {
        '$match': {
            'student_details': {'$ne': [] }
        }
    }
]

results = pd.DataFrame(courses_collection.aggregate(pipeline))
print(list(courses_collection.aggregate(pipeline)))
results

[{'_id': ObjectId('691346891e2873a74577e0f7'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_details': [{'_id': ObjectId('691346d51e2873a74577e115'), 'student_id': 1, 'name': 'Julioez', 'course': 'Math 101'}]}, {'_id': ObjectId('691346891e2873a74577e0fb'), 'course': 'CS 203', 'enrollments': 40, 'department': 'Computer Science', 'student_details': [{'_id': ObjectId('691346d51e2873a74577e116'), 'student_id': 2, 'name': 'Candita', 'course': 'CS 203'}]}, {'_id': ObjectId('691346891e2873a74577e0fe'), 'course': 'Physics 201', 'enrollments': 22, 'department': 'Physics', 'student_details': [{'_id': ObjectId('691346d51e2873a74577e117'), 'student_id': 3, 'name': 'Haga', 'course': 'Physics 201'}]}, {'_id': ObjectId('691346891e2873a74577e108'), 'course': 'Economics 202', 'enrollments': 33, 'department': 'Economics', 'student_details': [{'_id': ObjectId('691346d51e2873a74577e119'), 'student_id': 5, 'name': 'Latupeirissa', 'course': 'Economics 202'}]}, {'_id': Object

,_id,course,enrollments,department,student_details
0,691346891e2873a74577e0f7,Math 101,30,Mathematics,"[{'_id': 691346d51e2873a74577e115, 'student_id..."
1,691346891e2873a74577e0fb,CS 203,40,Computer Science,"[{'_id': 691346d51e2873a74577e116, 'student_id..."
2,691346891e2873a74577e0fe,Physics 201,22,Physics,"[{'_id': 691346d51e2873a74577e117, 'student_id..."
3,691346891e2873a74577e108,Economics 202,33,Economics,"[{'_id': 691346d51e2873a74577e119, 'student_id..."
4,691346891e2873a74577e10a,Engineering 202,25,Information Technology,"[{'_id': 691346d51e2873a74577e118, 'student_id..."
